# Crawling Code

In [1]:
# install GOT3 package
!pip install GetOldTweets3

In [1]:
# module import
import GetOldTweets3 as got
from bs4 import BeautifulSoup
import sys
import urllib
import json
import datetime
import time
import os
from random import uniform
from tqdm import tqdm
import csv
import pandas as pd

In [30]:
dateRange = pd.date_range(start='20150101', end='20200601', freq='MS')
for date in dateRange[:3]:
    print(set_crawl_date(date))

('2015-01-01', '2015-01-02')
('2015-02-01', '2015-02-02')
('2015-03-01', '2015-03-02')


In [33]:
for date in dateRange:
    crawl_start, crawl_end = set_crawl_date(date, 1)
    print(crawl_start, crawl_end)
    

트윗 수집 날짜 설정: 2015-01-01부터 2015-01-02까지
2015-01-01 2015-01-02
트윗 수집 날짜 설정: 2015-02-01부터 2015-02-02까지
2015-02-01 2015-02-02
트윗 수집 날짜 설정: 2015-03-01부터 2015-03-02까지
2015-03-01 2015-03-02
트윗 수집 날짜 설정: 2015-04-01부터 2015-04-02까지
2015-04-01 2015-04-02
트윗 수집 날짜 설정: 2015-05-01부터 2015-05-02까지
2015-05-01 2015-05-02
트윗 수집 날짜 설정: 2015-06-01부터 2015-06-02까지
2015-06-01 2015-06-02
트윗 수집 날짜 설정: 2015-07-01부터 2015-07-02까지
2015-07-01 2015-07-02
트윗 수집 날짜 설정: 2015-08-01부터 2015-08-02까지
2015-08-01 2015-08-02
트윗 수집 날짜 설정: 2015-09-01부터 2015-09-02까지
2015-09-01 2015-09-02
트윗 수집 날짜 설정: 2015-10-01부터 2015-10-02까지
2015-10-01 2015-10-02
트윗 수집 날짜 설정: 2015-11-01부터 2015-11-02까지
2015-11-01 2015-11-02
트윗 수집 날짜 설정: 2015-12-01부터 2015-12-02까지
2015-12-01 2015-12-02
트윗 수집 날짜 설정: 2016-01-01부터 2016-01-02까지
2016-01-01 2016-01-02
트윗 수집 날짜 설정: 2016-02-01부터 2016-02-02까지
2016-02-01 2016-02-02
트윗 수집 날짜 설정: 2016-03-01부터 2016-03-02까지
2016-03-01 2016-03-02
트윗 수집 날짜 설정: 2016-04-01부터 2016-04-02까지
2016-04-01 2016-04-02
트윗 수집 날짜 설정: 2016-05-01부

In [31]:
# setUntil : 마지막 날짜 배제
def set_crawl_date(start_date, freq=1):    
    end_date = start_date + datetime.timedelta(days=freq)
    
    # timestamp to string format
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    
    # check
    print("트윗 수집 날짜 설정: {0}부터 {1}까지".format(start_date, end_date))  
    
    return start_date, end_date

In [2]:
# GOT 스태틱 메소드 수정
def getJsonResponse(tweetCriteria, refreshCursor, cookieJar, proxy, useragent=None, debug=False):
    url = "https://twitter.com/i/search/timeline?"

    if not tweetCriteria.topTweets:
        url += "f=tweets&"

    url += ("vertical=news&q=%s&src=typd&%s"
            "&include_available_features=1&include_entities=1&max_position=%s"
            "&reset_error_state=false")

    urlGetData = ''
    
    # url + query search, since, since, until, lang    
    urlGetData += tweetCriteria.querySearch
    urlGetData += ' since:' + tweetCriteria.since
    urlGetData += ' until:' + tweetCriteria.until
    urlLang = 'l=' + tweetCriteria.lang + '&'
    
    # url 설정
    url = url % (urllib.parse.quote(urlGetData.strip()), urlLang, urllib.parse.quote(refreshCursor))
    useragent = useragent or TweetManager.user_agents[0]

    headers = [
        ('Host', "twitter.com"),
        ('User-Agent', useragent),
        ('Accept', "application/json, text/javascript, */*; q=0.01"),
        ('Accept-Language', "en-US,en;q=0.5"),
        ('X-Requested-With', "XMLHttpRequest"),
        ('Referer', url),
        ('Connection', "keep-alive")
    ]

    if proxy:
        opener = urllib.request.build_opener(urllib.request.ProxyHandler({'http': proxy, 'https': proxy}), urllib.request.HTTPCookieProcessor(cookieJar))
    else:
        opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookieJar))
    opener.addheaders = headers        
    
    time.sleep(1) # HTTP Request 429 에러 방지.

    ##################### 에러 핸들링 #####################
    # TimeOut 에러
    try:
        response = opener.open(url)
        jsonResponse = response.read()
    except TimeoutError as e:
        if debug: # 디버그 옵션
            print("에러 url 주소:", url)            
        print("Timeout error")
        print("30초 정지")
        time.sleep(30)

        # 한 번 더 시도
        try:
            response = opener.open(url)
            jsonResponse = response.read()
        except TimeoutError as e:
            print("Timeout error again. 패스.")
            pass

    # UrlParse 에러: 다시 시도 X
    except Exception as e:
        if debug: # 디버그 옵션
            print("에러 url 주소:", url)
            
        print("HTTP 요청 오류", str(e))        
        print("브라우저 오픈: https://twitter.com/search?q=%s&src=typd" % urllib.parse.quote(urlGetData))
        print("30초 정지")
        
        pass

    # Json 데이터 오류
    try:
        s_json = jsonResponse.decode()
    except:
        print("올바르지 못한 응답")
        if debug: # 디버그 옵션
            print("에러 url 주소:", url)
        pass
    else:
        try:
            dataJson = json.loads(s_json)
        except: # json 데이터 파싱 오류
            if debug: # 디버그 옵션
                print("에러 url 주소:", url)                
            print("JSON: %s" % s_json)
        pass

    return dataJson

In [3]:
# 커스텀 에러
class NotValidEndDateError(Exception):
    def __init__(self):
        super().__init__('마지막 검색 날짜를 다시 설정하십시오.')

In [4]:
# setUntil : 마지막 날짜 배제


def set_crawl_date(start_date, freq):
    start_date = datetime.datetime.strptime(str(start_date), "")

# def set_crawl_date(start_date, end_date):
    
#     start_date = datetime.datetime.strptime(str(start_date), "%Y%m%d")
#     end_date = datetime.datetime.strptime(str(end_date), "%Y%m%d") - datetime.timedelta(days=1)
    
#     if end_date == start_date:
#         raise NotValidEndDateError
    
#     else:   
#         print("트윗 수집 날짜 설정: {0}부터 {1}까지".format(start_date, end_date))    
#         return start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")

In [6]:
# got tweet 객체로부터 결과 추출
def get_results(tweet_data):
    results = []
    for tweet in tqdm(tweet_data):
        results.append({'url': tweet.permalink,
                        'date': tweet.date,
                        'text': tweet.text,
                        'user': tweet.username,
                        'mentions': tweet.mentions,
                        'retweets': tweet.retweets,
                        'favorites': tweet.favorites,
                        'hashtags': tweet.hashtags})
    return results        

In [7]:
# 추출한 결과 저장
def save_tweets(tweet_lists, base_file_dir="tweets"):
    
    if not os.path.exists(base_file_dir):
        os.makedirs(base_file_dir)
        
    with open(f"{base_file_dir}/tweets_{crawl_start}_{crawl_end}.csv", "a", -1, encoding="utf-8") as f:    
        writer = csv.writer(f)
        writer.writerow(['url', 'date', 'text', 'user', 'mentions', 'retweets', 'favorites', 'hashtags'])        
        for tweet_list in tqdm(tweet_lists):
            writer.writerow(list(tweet_list.values()))
            
    return 

In [9]:
# 크롤링 진행
crawl_start, crawl_end = set_crawl_date(20150601, 20150603)
tweet_results = crawl_tweets(crawl_start, crawl_end)
tweet_results_lists = get_results(tweet_results)
save_tweets(tweet_results_lists)

트윗 수집 날짜 설정: 2015-06-01 00:00:00부터 2015-06-02 00:00:00까지
========== 트윗 수집 시작: 2015-06-01 ~ 2015-06-02 ==========


100%|███████████████████████████████████████████████████████████████████████████| 2054/2054 [00:00<00:00, 85733.48it/s]

수집 완료 : 00:02:37
총 수집 트윗 개수 : 2054


In [ ]:
import abc
class AbstractCrawler(object):
    
    def __init__(self, url=None, default_headers= None, request_class=None, parser_class=None, config=None, **kwargs):
        self.base_url = url
    
    def _crawl(self, param=None, headers=None):
        url = self.make_url(self.base_url, param)
        header = self.__make_headers(headers)
        resp = request_class(url, header=header).get_data()
        pass
    
    @abc.abstractmethod
    def crawl(self, param, headers):
        return self._crawl(param, headers)
    
    def __make_headers(self,headers):
        return self.default_headers.update(headers)
    
    @abc.abstractmethod
    def __make_url(self, base_url, params):
        pass
        
    
    @abc.abstractmethod
    def run(self):
        data = self.crawl()
        self.save(data)
        pass
    
    @abc.abstractmethod
    def save(self):
        # db handling
        pass
